In [13]:
import cv2 as cv
import numpy as np

#### 1 Rotate image by 45 degrees without cropping the sides of the image. (Hint: There are 2 strategies to tackle these problems). Use "lena.jfif" as the input image.
- Use external libraries imutils.
- Modify the transformation matrix.

In [14]:
# method 1
lena = cv.imread("images/lena.jfif")
h, w = lena.shape[:2]
center = (w/2, h/2)
M = cv.getRotationMatrix2D(center, 45, 1.0)

# Rotated image dimension
abs_cos = abs(M[0, 0])
abs_sin = abs(M[0, 1])
new_width = int(h * abs_sin + w * abs_cos)
new_height = int(h * abs_cos + w * abs_sin)

M[0, 2] += (new_width / 2) - center[0]
M[1, 2] += (new_height / 2) - center[1]

rotated_lena = cv.warpAffine(lena, M, (new_width, new_height))
cv.imshow("rotated", rotated_lena)
cv.waitKey(0)
cv.destroyAllWindows()


In [15]:
import imutils
rotated = imutils.rotate_bound(lena, 45)
cv.imshow("Rotated (imutils)", rotated)
cv.waitKey(0)
cv.destroyAllWindows()

#### 2 Use the images with titles: "flower.jfif" and "native-bee.png". I want to put flower above an image. If I add two images, it will change color. If I blend it, I get a transparent effect. But I want it to be opaque. If it was a rectangular region, we could use the ROI as we did in the previous section. But flower is not a rectangular region. This is where bitwise operations, like AND, OR, NOT and XOR really come in handy. The associated functions are cv.bitwise_and(), cv.bitwise_or() and cv.bitwise_not(). You need to use cv.threshold function to segment the flower. Please refer to online documentation for more info. The result should resemble the following:

In [22]:
# Read the images
flower = cv.imread("images/flower.jfif", cv.IMREAD_UNCHANGED)
bee = cv.imread("images/native-bee.png", cv.IMREAD_UNCHANGED)

flower_gray = cv.cvtColor(flower, cv.COLOR_BGR2GRAY)
_, mask = cv.threshold(flower_gray, 75, 255, cv.THRESH_BINARY)

mask_inv = cv.bitwise_not(mask)

top_left_x = 0 
top_left_y = 0  

flower_height, flower_width = flower.shape[:2]
roi = bee[top_left_y:top_left_y + flower_height, top_left_x:top_left_x + flower_width].copy()

flower_fg = cv.bitwise_and(flower[..., :3], flower[..., :3], mask=mask)
bee_bg = cv.bitwise_and(roi[..., :3], roi[..., :3], mask=mask_inv)

if bee_bg.shape[2] != flower_fg.shape[2]:
    bee_bg = cv.cvtColor(bee_bg, cv.COLOR_BGR2BGRA)

result_roi = cv.add(bee_bg, flower_fg)
result_roi_bgra = cv.cvtColor(result_roi, cv.COLOR_BGR2BGRA)
bee[top_left_y:top_left_y + flower_height, top_left_x:top_left_x + flower_width] = result_roi_bgra

cv.imshow("Final result", bee)
cv.waitKey(0)
cv.destroyAllWindows()

#### 3 Write a function that randomly crop the central region of an image. The method signature should be as shown in the following:
random_center_crop(image, min_crop_ratio, max_crop_ratio)

In [23]:
def random_center_crop(image, min_crop_ratio, max_crop_ratio):
    
    if min_crop_ratio < 0.0 or min_crop_ratio > 1.0:
        raise ValueError("min_crop_ratio should be between 0.0 and 1.0")
    if max_crop_ratio < 0.0 or max_crop_ratio > 1.0:
        raise ValueError("max_crop_ratio should be between 0.0 and 1.0")
    if min_crop_ratio > max_crop_ratio:
        raise ValueError("min_crop_ratio should not be greater than max_crop_ratio")

    # Get image dimensions
    height, width = image.shape[:2]

    # Calculate crop sizes based on ratios
    min_crop_size = int(min(height, width) * min_crop_ratio)
    max_crop_size = int(min(height, width) * max_crop_ratio)

    # Randomly select crop size
    crop_size = np.random.randint(min_crop_size, max_crop_size + 1)

    # Calculate crop region
    start_x = max(0, (width - crop_size) // 2)
    start_y = max(0, (height - crop_size) // 2)
    end_x = start_x + crop_size
    end_y = start_y + crop_size

    # Perform crop
    cropped_image = image[start_y:end_y, start_x:end_x]

    return cropped_image

cropped = random_center_crop(lena, 0.5, 0.5)
cv.imshow("cropped", cropped)
cv.waitKey(0)
cv.destroyAllWindows()

#### 4 Aside from Gaussian noise, name another common type of noise. Write the code to demonstrate how the noise can be included in an image.

In [24]:
def add_salt_and_pepper_noise(image, salt_prob, pepper_prob):
    noisy = np.copy(image)
    
    # Salt noise
    salt = np.random.random(image.shape[:2]) < salt_prob
    noisy[salt] = 255
    
    # Pepper noise
    pepper = np.random.random(image.shape[:2]) < pepper_prob
    noisy[pepper] = 0
    
    return noisy

noisy_img = add_salt_and_pepper_noise(lena, salt_prob=0.02, pepper_prob=0.02)

cv.imshow('Original Image', lena)
cv.imshow('Image with Salt and Pepper Noise', noisy_img)

cv.waitKey(0)
cv.destroyAllWindows()
